<a href="https://colab.research.google.com/github/AmedeeRoy/BirdDL/blob/main/projection/SV_unet_projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# mount google drive
from google.colab import drive
drive.mount("/content/drive/")

%cd drive/My\ Drive/


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/'
/content/drive/My Drive


# LOAD MODELS

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

import sqlite3

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from utils.trip import *
from utils.unet import *

# Load data

In [32]:
data_all = pd.read_table('./data/gps+5-30s.csv', sep = ',')
check = pd.read_table('./data/check_gps+5-30s.csv', sep = ',')

# SV 5s
## data

In [33]:
select = check.trip[(check.species == 'SV') & (check.res == 5)]
data = data_all[data_all.trip.isin(select)].copy()
data['dive'] = 0
data = standardize_data(data)
data

,trip,datetime,lon,lat,gaps,step_speed,step_direction,dive,lon_std,lat_std,step_speed_std,step_direction_cos,step_direction_sin
460010,G1107_12_SV_T1,2007-11-19 13:42:28,-78.958106,-8.567359,0.0,8.306816,14.473134,0,1.917377,2.293521,0.412817,0.968265,0.249926
460011,G1107_12_SV_T1,2007-11-19 13:42:33,-78.958196,-8.567769,0.0,9.340164,10.728200,0,1.916609,2.289293,0.464307,0.982521,0.186150
460012,G1107_12_SV_T1,2007-11-19 13:42:38,-78.958359,-8.568248,0.0,11.251253,6.351008,0,1.915217,2.284353,0.559533,0.993863,0.110619
460013,G1107_12_SV_T1,2007-11-19 13:42:43,-78.958489,-8.568707,0.0,10.611651,-2.952198,0,1.914107,2.279619,0.527663,0.998673,-0.051503
460014,G1107_12_SV_T1,2007-11-19 13:42:48,-78.958627,-8.569070,0.0,8.633423,4.956724,0,1.912928,2.275875,0.429091,0.996260,0.086403
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1743485,P1115_8_SV_T1,2015-11-24 20:26:29,-77.264954,-11.771832,0.0,12.773153,-7.029047,0,1.783415,-0.308672,0.628083,0.992484,-0.122373
1743486,P1115_8_SV_T1,2015-11-24 20:26:34,-77.264771,-11.772295,0.0,11.052233,53.069752,0,1.785714,-0.311472,0.543461,0.600842,0.799368
1743487,P1115_8_SV_T1,2015-11-24 20:26:39,-77.264969,-11.772734,0.0,10.683502,44.976441,0,1.783227,-0.314127,0.525330,0.707397,0.706816
1743488,P1115_8_SV_T1,2015-11-24 20:26:44,-77.264557,-11.773157,0.0,13.011769,-67.459820,0,1.788403,-0.316685,0.639816,0.383331,-0.923611


In [34]:
model = torch.load('SV_unet_matrixlonlatcov_5s.pt')
window = 20
variable = ('lon_std', 'lat_std', 'gaps')

In [35]:
dive_estim = []

for i in select:
    t = data[data.trip == i].copy()
        
    test_set = TrajDataSet(t, window, variable, transform = ToTensor())
    
    # Test the model
    estim = np.zeros(len(t))
    nb = np.zeros(len(t))
    
    list_out = []
    model.eval()
    k = 0
    with torch.no_grad():
        for (x, y, z) in test_set:

            # Run the forward pass
            out = model(x, y.unsqueeze(1))
            
            estim[k:k + round(window)] += out.cpu().squeeze().numpy()
            nb[k:k + round(window)] += 1
            k+=1
    # # add to list by trajectory
    dive_estim.append(estim/nb)
    print(i)

data['prediction'] = 1/(1+np.exp(-np.hstack(dive_estim)))

G1107_12_SV_T1
G1107_14_SV_T1
G1107_15_SV_T1
G1107_17_SV_T1
G1107_18_SV_T1
G1107_18_SV_T2
G1107_19_SV_T1
G1107_19_SV_T2
G1107_20_SV_T1
G1107_21_SV_T1
G1107_22_SV_T1
G1107_24_SV_T1
G1107_25_SV_T1
G1107_26_SV_T1
G1107_27_SV_T1
G1107_27_SV_T2
G1107_31_SV_T1
G1107_34_SV_T1
G1107_34_SV_T2
G1107_34_SV_T3
G1107_35_SV_T1
G1107_35_SV_T2
G1107_36_SV_T1
G1107_37_SV_T1
G1107_37_SV_T2
G1107_39_SV_T1
G1107_4_SV_T1
G1107_40_SV_T1
G1107_40_SV_T2
G1107_41_SV_T1
G1107_42_SV_T1
G1107_42_SV_T2
G1107_43_SV_T1
G1107_45_SV_T1
G1107_46_SV_T1
G1107_47_SV_T1
G1107_49_SV_T1
G1107_49_SV_T2
G1107_5_SV_T1
G1107_50_SV_T1
G1107_52_SV_T1
G1107_53_SV_T1
G1107_54_SV_T1
G1107_55_SV_T1
G1107_58_SV_T1
G1107_7_SV_T1
G1107_7_SV_T2
G1107_8_SV_T1
G1107_9_SV_T1
P1108_1_SV_T1
P1108_1_SV_T2
P1108_11_SV_T1
P1108_11_SV_T2
P1108_12_SV_T1
P1108_18_SV_T1
P1108_18_SV_T2
P1108_2_SV_T1
P1108_2_SV_T2
P1108_2_SV_T3
P1108_22_SV_T1
P1108_22_SV_T2
P1108_22_SV_T3
P1108_26_SV_T1
P1108_28_SV_T1
P1108_28_SV_T2
P1108_29_SV_T1
P1108_29_SV_T2
P1108_

In [36]:
data.to_csv('SV_5s.csv', index = False)

# SV 15s
## data

In [37]:
select = check.trip[(check.species == 'SV') & (check.res == 15)]
data = data_all[data_all.trip.isin(select)].copy()
data['dive'] = 0
data = standardize_data(data)
data

,trip,datetime,lon,lat,gaps,step_speed,step_direction,dive,lon_std,lat_std,step_speed_std,step_direction_cos,step_direction_sin
793158,P1110_23_SV_T1,2010-11-26 14:12:26,-77.267321,-11.768224,0.0,17.585393,12.944815,0,1.386083,-1.711989,0.892469,0.974586,0.224012
793159,P1110_23_SV_T1,2010-11-26 14:12:41,-77.267353,-11.765576,0.0,19.648068,18.478843,0,1.385275,-1.686025,1.000000,0.948441,0.316954
793160,P1110_23_SV_T1,2010-11-26 14:12:56,-77.267469,-11.763126,0.0,18.204613,-1.950549,0,1.382425,-1.661992,0.924750,0.999421,-0.034037
793161,P1110_23_SV_T1,2010-11-26 14:13:11,-77.266847,-11.760942,0.0,16.828267,18.226772,0,1.397760,-1.640569,0.852998,0.949826,0.312779
793162,P1110_23_SV_T1,2010-11-26 14:13:26,-77.265835,-11.758832,0.0,17.293255,9.567553,0,1.422692,-1.619882,0.877239,0.986090,0.166210
...,...,...,...,...,...,...,...,...,...,...,...,...,...
985399,P1110_61_SV_T3,2010-12-05 22:58:59,-77.267330,-11.774556,0.0,10.115505,-17.482097,0,2.142882,-0.686498,0.503859,0.953811,-0.300408
985400,P1110_61_SV_T3,2010-12-05 22:59:14,-77.267123,-11.775805,0.0,9.391768,-9.275369,0,2.144069,-0.715970,0.467478,0.986925,-0.161180
985401,P1110_61_SV_T3,2010-12-05 22:59:29,-77.266502,-11.776443,0.0,6.538257,-34.415589,0,2.147630,-0.731014,0.324036,0.824960,-0.565191
985402,P1110_61_SV_T3,2010-12-05 22:59:44,-77.265556,-11.776324,0.0,6.926927,-53.683684,0,2.153052,-0.728210,0.343574,0.592243,-0.805760


In [38]:
model = torch.load('SV_unet_matrixlonlatcov_15s.pt')
window = 20
variable = ('lon_std', 'lat_std', 'gaps')

In [39]:
dive_estim = []

for i in select:
    t = data[data.trip == i].copy()
        
    test_set = TrajDataSet(t, window, variable, transform = ToTensor())
    
    # Test the model
    estim = np.zeros(len(t))
    nb = np.zeros(len(t))
    
    list_out = []
    model.eval()
    k = 0
    with torch.no_grad():
        for (x, y, z) in test_set:

            # Run the forward pass
            out = model(x, y.unsqueeze(1))
            
            estim[k:k + round(window)] += out.cpu().squeeze().numpy()
            nb[k:k + round(window)] += 1
            k+=1
    # # add to list by trajectory
    dive_estim.append(estim/nb)
    print(i)

data['prediction'] = 1/(1+np.exp(-np.hstack(dive_estim)))

P1110_23_SV_T1
P1110_23_SV_T2
P1110_23_SV_T3
P1110_27_SV_T1
P1110_27_SV_T2
P1110_37_SV_T1
P1110_37_SV_T2
P1110_37_SV_T3
P1110_57_SV_T1
P1110_57_SV_T2
P1110_61_SV_T1
P1110_61_SV_T2
P1110_61_SV_T3


In [40]:
data.to_csv('SV_15s.csv', index = False)

# SV 30s
## data

In [41]:
select = check.trip[(check.species == 'SV') & (check.res == 30)]
data = data_all[data_all.trip.isin(select)].copy()
data['dive'] = 0
data = standardize_data(data)
data

,trip,datetime,lon,lat,gaps,step_speed,step_direction,dive,lon_std,lat_std,step_speed_std,step_direction_cos,step_direction_sin
458645,G1107_1_SV_T1,2007-11-17 10:12:23,-78.966934,-8.569705,0.000000,7.791539,-7.744152,0,-2.145905,1.742132,0.361735,0.990880,-0.134750
458646,G1107_1_SV_T1,2007-11-17 10:12:53,-78.967828,-8.571593,0.000000,7.735141,6.693300,0,-2.164214,1.731126,0.359108,0.993184,0.116555
458647,G1107_1_SV_T1,2007-11-17 10:13:23,-78.970012,-8.573651,0.000000,11.068354,21.302796,0,-2.208967,1.719130,0.514374,0.931674,0.363297
458648,G1107_1_SV_T1,2007-11-17 10:13:53,-78.971326,-8.575501,0.000000,8.388760,-11.280612,0,-2.235906,1.708348,0.389555,0.980681,-0.195614
458649,G1107_1_SV_T1,2007-11-17 10:14:23,-78.972506,-8.577332,0.000000,8.053516,-2.586444,0,-2.260082,1.697679,0.373939,0.998981,-0.045127
...,...,...,...,...,...,...,...,...,...,...,...,...,...
796113,P1110_24_SV_T4,2010-11-28 19:45:50,-77.270959,-11.780975,0.000000,17.241997,1.555730,0,1.822496,1.643211,0.871413,0.999631,0.027149
796114,P1110_24_SV_T4,2010-11-28 19:46:20,-77.267834,-11.777413,0.000000,17.421032,-0.237631,0,1.881699,1.732217,0.880746,0.999991,-0.004147
796115,P1110_24_SV_T4,2010-11-28 19:46:50,-77.264863,-11.777193,0.483333,10.822186,45.027780,0,1.937991,1.737706,0.536774,0.706764,0.707450
796116,P1110_24_SV_T4,2010-11-28 19:47:20,-77.264795,-11.775654,1.000000,5.716652,-83.197578,0,1.939283,1.776167,0.270643,0.118446,-0.992961


In [42]:
model = torch.load('SV_unet_matrixlonlatcov_30s.pt')
window = 20
variable = ('lon_std', 'lat_std', 'gaps')

In [43]:
dive_estim = []

for i in select:
    t = data[data.trip == i].copy()
        
    test_set = TrajDataSet(t, window, variable, transform = ToTensor())
    
    # Test the model
    estim = np.zeros(len(t))
    nb = np.zeros(len(t))
    
    list_out = []
    model.eval()
    k = 0
    with torch.no_grad():
        for (x, y, z) in test_set:

            # Run the forward pass
            out = model(x, y.unsqueeze(1))
            
            estim[k:k + round(window)] += out.cpu().squeeze().numpy()
            nb[k:k + round(window)] += 1
            k+=1
    # # add to list by trajectory
    dive_estim.append(estim/nb)
    print(i)

data['prediction'] = 1/(1+np.exp(-np.hstack(dive_estim)))

G1107_1_SV_T1
G1107_1_SV_T2
G1107_10_SV_T1
G1107_10_SV_T2
G1107_11_SV_T1
G1107_2_SV_T1
G1107_23_SV_T1
G1107_30_SV_T1
G1107_30_SV_T2
G1107_33_SV_T1
G1107_33_SV_T2
G1107_44_SV_T1
G1107_56_SV_T1
P1108_10_SV_T1
P1108_13_SV_T1
P1108_13_SV_T2
P1108_14_SV_T1
P1108_14_SV_T2
P1108_14_SV_T3
P1108_15_SV_T1
P1108_19_SV_T1
P1108_19_SV_T2
P1108_20_SV_T1
P1108_20_SV_T2
P1108_20_SV_T3
P1108_20_SV_T4
P1108_21_SV_T1
P1108_21_SV_T2
P1108_21_SV_T3
P1108_24_SV_T1
P1108_24_SV_T2
P1108_24_SV_T3
P1108_24_SV_T4
P1108_25_SV_T1
P1108_25_SV_T2
P1108_25_SV_T3
P1108_25_SV_T4
P1108_27_SV_T1
P1108_27_SV_T2
P1108_27_SV_T3
P1108_27_SV_T4
P1108_27_SV_T5
P1108_27_SV_T6
P1108_30_SV_T1
P1108_30_SV_T2
P1108_30_SV_T3
P1108_34_SV_T1
P1108_34_SV_T2
P1108_34_SV_T3
P1108_34_SV_T4
P1108_34_SV_T5
P1108_37_SV_T1
P1108_37_SV_T2
P1108_37_SV_T3
P1108_38_SV_T1
P1108_38_SV_T2
P1108_38_SV_T3
P1108_38_SV_T4
P1108_38_SV_T5
P1108_39_SV_T1
P1108_39_SV_T2
P1108_40_SV_T1
P1108_40_SV_T2
P1108_40_SV_T3
P1108_40_SV_T4
P1108_41_SV_T1
P1108_41_SV_T

In [44]:
data.to_csv('SV_30s.csv', index = False)